# 模型评估和超参调试的最佳实践

- 获取模型性能的无偏估计
- 诊断机器学习模型的常见问题
- 微调机器学习模型
- 使用不同的性能指标评估预测模型

## 使用管道简化工作流程


### 加载乳腺癌威斯康星数据集

In [5]:
# 1. 加载数据
import pandas as pd
df = pd.read_csv('wdbc.data', header=None)


In [11]:
# 2. 提取X和y,并且对y的值进行编码
from sklearn.preprocessing import LabelEncoder
X = df.iloc[:, 2:].values
y = df.iloc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)  # malignant=1,benign=0
le.transform(['M', 'B'])


array([1, 0], dtype=int64)

In [13]:
# 3. 数据集划分
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


### 在管道中结合transformers和estimators

In [15]:
# 在将数据喂给线性分类器前,我们需要1.将特征值进行标准化 2.对特征维度进行降维(通过PCA的方式)
# 我们使用管道的方式来代替分别对训练集和测试集进行上述动作
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', LogisticRegression(random_state=1))])
pipe_lr.fit(X_train, y_train)
print('Test Accuracy: {:.3}'.format(pipe_lr.score(X_test, y_test)))


Test Accuracy: 0.947


D:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
